In [ ]:
class DonutStand:
    """A class to model a donut stand offering a specific
    flavor of donuts."""

    def __init__(self, flavor: str) -> None:
        """Initialize a new stand with the specified flavor. This stand
        will eventually point to a next stand."""
        self._flavor = flavor
        self._next = None

    def __repr__(self) -> str:
        """Formal text representation"""
        return f"DonutStand({self._flavor!r})"

    def is_flavor(self, target: str) -> bool:
        """Asserts if a stand offers the specified flavor"""
        return target and target == self._flavor

In [ ]:
class DonutRun:
    """Simulates the Great Chicago Donut Run"""

    def __init__(self):
        # Class DonutRun has two fields: a pointer to the first
        # stand, and a count of stands in the run. The second
        # field, the count, was not specified explicitly but was
        # not prohibited either. The assignment can be completed
        # without a count field - I have left some loops below
        # in a count-free form to show how.
        self._head_stand = None
        self._count = 0

    def add_stand(self, flavor: str):
        """This method is not specified explicitly in the problem. It is, however,
        implied because how else are we going to build the explicitly specified
        methods?"""
        new_stand = DonutStand(flavor)
        # If the DonutRun object is empty, make this stand its first one, otherwis
        # make a judgement call where to place the stand.
        if self._head_stand is None:
            self._head_stand = new_stand
            # Because this is a circular structure we must point the
            # first and only stand back to itself
            self._head_stand._next = self._head_stand
        else:
            # Here's the judgement call: in an open-ended linkedlist usually we
            # add the new node at the end of the list; the location is known
            # either by traversing the list till we find the node whose next
            # pointer is null, or by referencing a linkedlist.last or
            # linkedlist.tail node. Here, we cannot traverse the circular
            # structure to find its last node because we'll get into an infinite
            # loop. Instead we insert any new node immediately after the
            # first node.
            remember_next = self._head_stand._next
            self._head_stand._next = new_stand
            new_stand._next = remember_next
            # Alternatively, we could have traversed this circular structure
            # until the cursor.next pointed to self._start, and inject the
            # new node between cursor and _start. That would have made additions
            # to the data structure O(n) instead of O(1) with the technique
            # we use above.
        # Update the count of stands
        self._count += 1

    def _count_stands(self, target_flavor=None) -> int:
        """Counts the number of stands in the donut run - if an argument
        is provided, the method counts the stands that have the specific
        flavor."""
        # Assume we are not counting specific flavors so just grab
        # the current count of stands in the object
        count = self._count
        # If a flavor has been specified and the Donut Run is not
        # empty, prepare to go around every stand
        if self._head_stand is not None and target_flavor is not None:
            stand = self._head_stand._next
            count = 1 if self._head_stand.is_flavor(target_flavor) else 0
            while stand != self._head_stand:
                if stand.is_flavor(target_flavor):
                    count += 1
                stand = stand.next
        return count

    def count_stands(self) -> int:
        """Counts all the stands by calling the internal counting method
        without specifying a flavor"""
        return self._count_stands()

    def count_flavor_stands(self, target_flavor) -> int:
        """Counts all the stands by calling the internal counting method
        by specifying a flavor"""
        return self._count_stands(target_flavor)

    def find(self, target_flavor):
        """Wrapper function for _search looking for specified flavor
        in current stand."""
        return self._search(target_flavor, before=False)

    def find_before(self, target_flavor):
        """Wrapper function for _search looking for specified flavor
        in next stand."""
        return self._search(target_flavor, before=True)

    def _search(self, target_flavor, before):
        """Internal function to find the first stand with the specified flavor
        or the stand before it."""
        # Initialize return object
        found_stand = None
        # Operate only on a non-empty Donut Run
        if self._count > 0:
            # Initialize a cursor
            stand = self._head_stand
            # Initialize a counter to avoid infinite loops
            steps = 0
            # Go around until all stands have been checked or
            # the desired stand is found
            while steps < self._count and found_stand is None:
                # The candidate stand is where we are looking for the
                # specified flavor: current stand or next stand,
                # depending on the before flag.
                if before:
                    candidate = stand._next
                else:
                    candidate = stand
                # Likewise, if we find a candidate stand with the
                # specified flavor, what shall we return: the stand
                # with the flavor or the previous stand?
                if candidate.is_flavor(target_flavor):
                    if before:
                        found_stand = stand
                    else:
                        found_stand = candidate
                # Check the next stand and update the counter
                stand = stand._next
                steps += 1
        # Done
        return found_stand

    # Constant with specific flavor for assignment methods
    _CHOCOLATE = "Chocolate"

    def find_chocolate(self):
        """Fulfills assignment specification to find the first chocolate stand."""
        return self.find(self._CHOCOLATE)

    def find_before_chocolate(self):
        """Fulfills assignment specification to find the stand before
        the first chocolate stand."""
        return self.find_before(self._CHOCOLATE)

    def insert_after_flavor(self, target_flavor, new_flavor):
        """Adds a stand after a specific stand"""
        if self._count == 0:
            # Special case, when the Donut Run is empty
            self.add_stand(new_flavor)
        else:
            # Prepare a new stand to insert
            new_stand = DonutStand(new_flavor)
            # special case: the head stand has our target flavor
            if self._head_stand.is_flavor(target_flavor):
                remember_next = self._head_stand._next
                self._head_stand._next = new_stand
                new_stand._next = remember_next
            else:
                # Traverse the run until we find the target flavor.
                # Note that there may not be a stand with that
                # target flavor and the traversal may end without a
                # successful insertion. If a new stand is added,
                # however, the traversal ends thanks to the boolean
                # flag "found_insertion_point"
                cursor = self._head_stand._next
                found_insertion_point = False
                while cursor != self._head_stand and not found_insertion_point:
                    found_insertion_point = cursor.is_flavor(target_flavor)
                    if found_insertion_point:
                        remember_next = cursor._next
                        cursor._next = new_stand
                        new_stand._next = remember_next
                    cursor = cursor._next